## Computer Vision Edge Deployment Demonstration

This notebook will walk through building Wallaroo pipeline with a a Classification model deployed to detect the likelihood of credit card fraud, then publishing that pipeline to an Open Container Initiative (OCI) Registry where it can be deployed in other Docker and Kubernetes environments.

This demonstration will focus on deployment to the edge.  For further examples of using Wallaroo with this computer vision models, see [Wallaroo 101](https://docs.wallaroo.ai/wallaroo-101/).

This demonstration will perform the following:

1. As a Data Scientist:
    1. Upload a computer vision model to Wallaroo, deploy it in a Wallaroo pipeline, then perform a sample inference.
    1. Publish the pipeline to an Open Container Initiative (OCI) Registry service.  This is configured in the Wallaroo instance.  See [Edge Deployment Registry Guide](https://staging.docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.  This demonstration uses a GitHub repository - see [Introduction to GitHub Packages](https://docs.github.com/en/packages/learn-github-packages/introduction-to-github-packages) for setting up your own package repository using GitHub, which can then be used with this tutorial.
    1. View the pipeline publish details.
1. As a DevOps Engineer:
    1. Deploy the published pipeline into an edge instance.  This example will use Docker.
    1. Perform a sample inference into the deployed pipeline with the same data used in the data scientist example.

## Data Scientist Pipeline Publish Steps

### Load Libraries

The first step is to import the libraries used in this notebook.

In [26]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import pandas as pd
import requests


# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=QHMK-HCLH

Login successful!


### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [3]:
import string
import random

# make a random 4 character suffix to verify uniqueness in tutorials
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='john'

workspace_name = f'edge-publish-demo{suffix}'
pipeline_name = 'edge-pipeline'
model_name = 'ccfraud'
model_file_name = './models/xgboost_ccfraud.onnx'

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [5]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'edge-publish-demojohn', 'id': 5, 'archived': False, 'created_by': 'db364f8c-b866-4865-96b7-0b65662cb384', 'created_at': '2023-08-24T16:57:29.048825+00:00', 'models': [], 'pipelines': []}

### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in ONNX format, which is specified in the `framework` parameter.

In [6]:
edge_demo_model = wl.upload_model(
    model_name,
    model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
)

### Reserve Pipeline Resources

Before deploying an inference engine we need to tell wallaroo what resources it will need.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be.

We will be testing this deployment for an edge scenario, so the resource specifications are kept small -- what's the minimum needed to meet the expected load on the planned hardware.

- cpus - 0.5 => allow the engine to use 4 CPU cores when running the neural net
- memory - 900Mi => each inference engine will have 2 GB of memory, which is plenty for processing a single image at a time.
- arch - we will specify the X86 architecture.

In [7]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("900Mi").build()

### Simulated Edge Deployment

We will now deploy our pipeline into the current Kubernetes environment using the specified resource constraints. This is a "simulated edge" deploy in that we try to mimic the edge hardware as closely as possible.

In [9]:
pipeline = get_pipeline(pipeline_name)
display(pipeline)

pipeline.add_model_step(edge_demo_model)

pipeline.deploy(deployment_config = deploy_config)

name,edge-pipeline
created,2023-08-24 16:57:29.486951+00:00
last_updated,2023-08-24 16:57:29.486951+00:00
deployed,(none)
tags,
versions,d0a55f2b-0938-45a0-ae58-7d78b9b590d6
steps,
published,False


Waiting for deployment - this will take up to 45s ......................... ok


name,edge-pipeline
created,2023-08-24 16:57:29.486951+00:00
last_updated,2023-08-24 16:57:40.183869+00:00
deployed,True
tags,
versions,"7c4383d2-b79d-4179-91a1-b592803e1373, d0a55f2b-0938-45a0-ae58-7d78b9b590d6"
steps,ccfraud
published,False


### Run Single Image Inference

A single image, encoded using the Apache Arrow format, is sent to the deployed pipeline. Arrow is used here because, as a binary protocol, there is far lower network and compute overhead than using JSON. The Wallaroo Server engine accepts both JSON, pandas DataFrame, and Apache Arrow formats.

The sample DataFrames and arrow tables are in the `./data` directory.  We'll use the Apache Arrow table `cc_data_10k.arrow`.

Once complete, we'll display how long the inference request took.

In [14]:
with pa.ipc.open_file("./data/cc_data_10k.arrow") as f:
    data = f.read_all()

results = pipeline.infer(data, dataset=["*", "metadata.elapsed"])
elapsed = results['metadata.elapsed'][0].as_py()[1] / 1000000.0

print(f"Elapsed: {elapsed} ms")

Elapsed: 8.292807 ms


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://staging.docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)` which has the following parameters and returns.

#### Publish a Pipeline Parameters

The `publish` method takes the following parameters.  The containerized pipeline will be pushed to the Edge registry service with the model, pipeline configurations, and other artifacts needed to deploy the pipeline.

| Parameter | Type | Description |
|---|---|---|
| `deployment_config` | `wallaroo.deployment_config.DeploymentConfig` (*Optional*) | Sets the pipeline deployment configuration.  For example:    For more information on pipeline deployment configuration, see the [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration]({{<ref "wallaroo-sdk-essentials-pipeline-deployment-config">}}).

#### Publish a Pipeline Returns

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline version id | integer | Numerical Wallaroo id of the pipeline version published. |
| status | string | The status of the pipeline publication.  Values include:  <ul><li>PendingPublish: The pipeline publication is about to be uploaded or is in the process of being uploaded.</li><li>Published:  The pipeline is published and ready for use.</li></ul> |
| Engine URL | string | The URL of the published pipeline engine in the edge registry. |
| Pipeline URL | string | The URL of the published pipeline in the edge registry. |
| Helm Chart URL | string | The URL of the helm chart for the published pipeline in the edge registry. |
| Helm Chart Reference | string | The help chart reference. |
| Helm Chart Version | string | The version of the Helm Chart of the published pipeline.  This is also used as the Docker tag. |
| Engine Config | `wallaroo.deployment_config.DeploymentConfig` | The pipeline configuration included with the published pipeline. |
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [17]:
pub=pipeline.publish(deploy_config)
pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing...Published.


ID,2
Pipeline Version,710aad65-1437-487b-b6db-0f732b5d2d73
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engine:v2023.3.0-main-3731
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:710aad65-1437-487b-b6db-0f732b5d2d73
Helm Chart URL,ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:02f46e1f2abe5797b35f666a1cef9c61fe884aead61e400ce3dae11611a30404
Helm Chart Version,0.0.1-710aad65-1437-487b-b6db-0f732b5d2d73
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 0.5, 'memory': '900Mi'}, 'requests': {'cpu': 0.5, 'memory': '900Mi'}}}, 'engineAux': {'images': {}}, 'enginelb': {}}"
Created By,john.hummel@wallaroo.ai
Created At,2023-08-24 17:05:01.156123+00:00


In [18]:
display(pipeline)

name,edge-pipeline
created,2023-08-24 16:57:29.486951+00:00
last_updated,2023-08-24 17:05:01.060493+00:00
deployed,True
tags,
versions,"710aad65-1437-487b-b6db-0f732b5d2d73, 44c71e77-d8fa-4015-aeee-1cdbccfeb0ef, 7c4383d2-b79d-4179-91a1-b592803e1373, d0a55f2b-0938-45a0-ae58-7d78b9b590d6"
steps,ccfraud
published,True


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [19]:
wl.list_pipelines()

name,created,last_updated,deployed,tags,versions,steps,published
edge-pipeline,2023-24-Aug 16:57:29,2023-24-Aug 17:05:01,True,,"710aad65-1437-487b-b6db-0f732b5d2d73, 44c71e77-d8fa-4015-aeee-1cdbccfeb0ef, 7c4383d2-b79d-4179-91a1-b592803e1373, d0a55f2b-0938-45a0-ae58-7d78b9b590d6",ccfraud,True


### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [20]:
pipeline.publishes()

id,pipeline_version_name,engine_url,pipeline_url,created_by,created_at,updated_at
1,44c71e77-d8fa-4015-aeee-1cdbccfeb0ef,None,None,john.hummel@wallaroo.ai,2023-24-Aug 16:59:07,2023-24-Aug 16:59:07
2,710aad65-1437-487b-b6db-0f732b5d2d73,ghcr.io/wallaroolabs/doc-samples/engine:v2023.3.0-main-3731,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:710aad65-1437-487b-b6db-0f732b5d2d73,john.hummel@wallaroo.ai,2023-24-Aug 17:05:01,2023-24-Aug 17:05:01


## DevOps Deployment

We now have our pipeline published to our Edge Registry service.  We can deploy this in a x86 environment running Docker that is logged into the same registry service that we deployed to.

For more details, check with the documentation on your artifact service.  The following are provided for the three major cloud services:

* [Set up authentication for Docker](https://cloud.google.com/artifact-registry/docs/docker/authentication)
* [Authenticate with an Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli)
* [Authenticating Amazon ECR Repositories for Docker CLI with Credential Helper](https://aws.amazon.com/blogs/compute/authenticating-amazon-ecr-repositories-for-docker-cli-with-credential-helper/)

### Docker Deployment

Using our sample environment, here's sample deployment of our pipeline using Docker.  The standard format is the following:

```bash
docker run -p 8080:8080 \
    -e DEBUG=true -e OCI_REGISTRY={your registry service} \
    -e CONFIG_CPUS=4 \
    -e OCI_USERNAME={registry username here} \
    -e OCI_PASSWORD={registry token here} \
    -e PIPELINE_URL={your pipeline deployment url} \
    {your engine url}
```

For example, our sample docker run is:

```bash
docker run -p 8080:8080 \
    -e DEBUG=true -e OCI_REGISTRY=ghcr.io \
    -e CONFIG_CPUS=4 \
    -e OCI_USERNAME={registry username here} \
    -e OCI_PASSWORD={registry token here} \
    -e PIPELINE_URL=ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:710aad65-1437-487b-b6db-0f732b5d2d73 \
    ghcr.io/wallaroolabs/doc-samples/engine:v2023.3.0-main-3731
```

### Docker Compose Deployment

A `docker compose` example would look like the following:

```yaml
services:
  engine:
    image: ghcr.io/wallaroolabs/doc-samples/engine:v2023.3.0-main-3731
    ports:
      - 8080:8080
    environment:
      PIPELINE_URL: ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:710aad65-1437-487b-b6db-0f732b5d2d73
      OCI_USERNAME: {registry username here}
      OCI_PASSWORD: {registry token here}
      OCI_REGISTRY: ghcr.io
      CONFIG_CPUS: 1
```

### Helm Deployment

Helm deployments will take two steps.

1. Pull the helm charts from the published pipeline.  The two fields are the Helm Chart URL and the Helm Chart version to specify the OCI .  For our example, that command is:

  ```bash
  helm pull oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline --version 0.0.1-710aad65-1437-487b-b6db-0f732b5d2d73
  ```

1. Extract the `tgz` file and copy the `values.yaml` and copy the values used to edit engine allocations, etc.  The following are **required** for the deployment to run:

  ```yml
  ociRegistry:
  registry: {your registry service}
  username:  {registry username here}
  password: {registry token here}
  ```

  Store this into another file, like `local-values.yaml`.

1. Create the namespace to deploy the pipeline to.  For example, the namespace `wallaroo-edge-pipeline` would be:

  ```bash
  kubectl create -n wallaroo-edge-pipeline
  ```

1. Deploy the `helm` installation with `helm install` through one of the following options:
    1. Specify the `tgz` file that was downloaded and the local values file.  For example:

      ```bash
      
      ```




## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `testboy.local`.  Replace this URL with the URL of you edge deployment.


In [9]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-pipeline","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

In [45]:
!curl testboy.local:8080/models

{"models":[{"name":"ccfraud","sha":"054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52","status":"Running","version":"1d7eaf5d-ccd5-49e7-9ecc-13343e5af484"}]}

### Edge Deployed Inference

The inference endpoint takes the following pattern:

* `/pipelines/{pipeline-name}`:  The `pipeline-name` is the same as returned from the [`/pipelines`](#list-pipelines) endpoint as `id`.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.
* `Content-Type: application/json`: JSON.

The `Accept` header will determine what kind format of the data is returned.

* `Accept: application/json`: Returns a JSON object in the following format.

* **check_failures** (*List[Integer]*): Any Validations that failed.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://staging.docs.wallaroo.ai/20230201/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing)
* **elasped** (*List[Integer]*): A List of time in nanoseconds for:
  * [0]: The time to serialize the input.
  * [1...n]: How long each step took.
* **model_name** (*String*): The name of the model.
* **model_version** (*String*): The UUID identifier of the model version from Wallaroo.
* **original_data** (*List*): The original submitted data.
* **outputs** (*List*): A List of outputs with each output field corresponding to the input.  This is in the format for each data type returned:
  * **{data-type}**: The data type being returned.
    * **data** (*List*): The data from this data type.
    * **dim** (*List*): The shape of the data for this data type.
* **dim** (*List*): The shape of the data received.
* **pipeline_name** (*String*): The name of the pipeline.
* **shadow_data** (*List*): Any data returned from a shadow deployed model.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Shadow Deployments](https://staging.docs.wallaroo.ai/20230201/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#pipeline-shadow-deployments).
* **time** (*Integer*): The Epoch time of the inference.
  


In [53]:
!curl -X POST testboy.local:8080/pipelines/edge-pipeline -H "Content-Type: application/vnd.apache.arrow.file" --data-binary @./data/cc_data_10k.arrow

[{"check_failures":[],"elapsed":[1795547,2734591],"model_name":"ccfraud","model_version":"1d7eaf5d-ccd5-49e7-9ecc-13343e5af484","original_data":null,"outputs":[{"Float":{"data":[1.0094895362854004,1.0094895362854004,1.0094895362854004,1.0094895362854004,-1.8477439880371094e-6,-0.00004494190216064453,-0.00009369850158691406,-0.0000832676887512207,-0.00008344650268554688,0.0004897117614746094,0.0006609857082366943,0.00007578730583190918,-0.00010061264038085938,-0.000519871711730957,-3.7550926208496094e-6,-0.00010895729064941406,-0.00017696619033813477,-0.000028371810913085938,0.00002181529998779297,-0.00008505582809448242,-0.000049233436584472656,0.00020524859428405762,0.00001677870750427246,-0.00008910894393920898,0.00008806586265563965,0.0002167224884033203,-0.00003045797348022461,-0.0001615285873413086,-0.00007712841033935547,-8.046627044677734e-6,-0.000048100948333740234,-0.00013583898544311523,-0.000014126300811767578,-0.00015842914581298828,-0.00004863739013671875,0.000073105096817

In [70]:
!curl -X POST testboy.local:8080/pipelines/edge-pipeline -H "Content-Type: application/json; format=pandas-records" --data-binary @./data/cc_data_10k.df.json

[{"check_failures":[],"elapsed":[100131380,4155504],"model_name":"ccfraud","model_version":"1d7eaf5d-ccd5-49e7-9ecc-13343e5af484","original_data":null,"outputs":[{"Float":{"data":[1.0094895362854004,1.0094895362854004,1.0094895362854004,1.0094895362854004,-1.8477439880371094e-6,-0.00004494190216064453,-0.00009369850158691406,-0.0000832676887512207,-0.00008344650268554688,0.0004897117614746094,0.0006609857082366943,0.00007578730583190918,-0.00010061264038085938,-0.000519871711730957,-3.7550926208496094e-6,-0.00010895729064941406,-0.00017696619033813477,-0.000028371810913085938,0.00002181529998779297,-0.00008505582809448242,-0.000049233436584472656,0.00020524859428405762,0.00001677870750427246,-0.00008910894393920898,0.00008806586265563965,0.0002167224884033203,-0.00003045797348022461,-0.0001615285873413086,-0.00007712841033935547,-8.046627044677734e-6,-0.000048100948333740234,-0.00013583898544311523,-0.000014126300811767578,-0.00015842914581298828,-0.00004863739013671875,0.0000731050968

Here's another version using the requests library.  We will submit the same Apache Arrow table for an inference request, and only show the first 10 rows of the outputs.

In [77]:
import json

# set the content type and accept headers
headers = {
    'Content-Type': 'application/vnd.apache.arrow.file'
}

# Submit arrow file
dataFile="./data/cc_data_10k.arrow"

data = open(dataFile,'rb').read()

host = 'http://testboy.local:8080'

deployurl = f'{host}/pipelines/edge-pipeline'

response = requests.post(
                    deployurl, 
                    headers=headers, 
                    data=data, 
                    verify=True
                )

# display the first 5 results back
display(response.json()[0]['outputs'][0]['Float']['data'][0:10])

[1.0094895362854004,
 1.0094895362854004,
 1.0094895362854004,
 1.0094895362854004,
 -1.8477439880371094e-06,
 -4.494190216064453e-05,
 -9.369850158691406e-05,
 -8.32676887512207e-05,
 -8.344650268554688e-05,
 0.0004897117614746094]

We'll do the same thing, only this time submit the data in the form of pandas record format.

In [78]:
import json

# set the content type and accept headers
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

# Submit arrow file
dataFile="./data/cc_data_10k.df.json"

data = json.load(open(dataFile))

host = 'http://testboy.local:8080'

deployurl = f'{host}/pipelines/edge-pipeline'

response = requests.post(
                    deployurl, 
                    headers=headers, 
                    json=data, 
                    verify=True
                )

# # display the first 5 results back
display(response.json()[0]['outputs'][0]['Float']['data'][0:10])


[1.0094895362854004,
 1.0094895362854004,
 1.0094895362854004,
 1.0094895362854004,
 -1.8477439880371094e-06,
 -4.494190216064453e-05,
 -9.369850158691406e-05,
 -8.32676887512207e-05,
 -8.344650268554688e-05,
 0.0004897117614746094]